In [1]:
!pip install librosa
!pip install resampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.4 MB/s eta 0:00:00


In [2]:
colab = True

In [4]:
if colab:
    !pip install SoundFile
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    import zipfile

    # Google Authentication
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # Download code files
    downloaded = drive.CreateFile({'id':"1js1fz4Bu98NIds4cErL7NdT1efxavixp"})
    downloaded.GetContentFile('Code.zip')

    # Extract files to temporary location in Google Drive
    with zipfile.ZipFile('Code.zip', 'r') as zip_file:
        zip_file.extractall()

    # Download data files
    downloaded = drive.CreateFile({'id':"1seDpWl9c28V-kCQxjhkuaws7i8d-_R31"})
    downloaded.GetContentFile('Data.zip')

    # Extract files to temporary location in Google Drive
    with zipfile.ZipFile('Data.zip', 'r') as zip_file:
        zip_file.extractall()

In [5]:
from Preprocessing import *
import librosa
import numpy as np
import random
import tensorflow as tf
from IPython.display import Audio

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
basedir = "gdrive/MyDrive/Deep Learning For Ecology/Bird Call Detection Model/"

Mounted at /content/gdrive


In [ ]:
positive_class = ['1'] # which labels should be bundled together for the positive  class
negative_class = ['0'] # which labels should be bundled together for the negative  class


# Data hyper-parameters
# -----------------------------
lowpass_cutoff = 2000 # Cutt off for low pass filter
downsample_rate = 4100 # Frequency to downsample to
nyquist_rate = 2050 # Nyquist rate (half of sampling rate)
segment_duration = 4 # how long should a segment be


# Spectrogram hyper-parameters
# -----------------------------
n_fft = 1024 # Hann window length
hop_length = 256 # Sepctrogram hop size
n_mels = 128 # Spectrogram number of mells
f_min = 4000 # Spectrogram, minimum frequency for call
f_max = 8000 # Spectrogram, maximum frequency for call

# Don't change these
# -----------------------------
species_folder = '.' # Should contain /Audio and /Annotations, don't change this
file_type = 'svl' # don't change this
audio_extension = '.wav' # don't change this

In [ ]:
# This takes about 1 minute
pre_pro = Preprocessing(species_folder, lowpass_cutoff,
                downsample_rate, nyquist_rate,
                segment_duration,
                positive_class, negative_class,n_fft,
                hop_length, n_mels, f_min, f_max, file_type,
                audio_extension)

X, Y = pre_pro.create_dataset(False)

In [7]:
x_file = basedir+'x.npy'
y_file = basedir+'y.npy'

with open(x_file, 'rb') as f:
    X = np.load(f)
with open(y_file, 'rb') as f:
    Y = np.load(f)

In [9]:
print(X.shape)
print(Y.shape)

(5782, 16400)
(5782,)


In [10]:
X = X.reshape(X.shape[0], X.shape[1], 1)

In [11]:
print(X.shape)
print(Y.shape)

(5782, 16400, 1)
(5782,)


In [14]:
!pip install tensorflow-hub

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

In [19]:
yamnet_model.layers

AttributeError: '_UserObject' object has no attribute 'layers'

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

yamnet_model = tf.keras.Sequential([
    hub.load('https://tfhub.dev/google/yamnet/1')
])


# Créer votre propre modèle en utilisant des parties de YAMNet
input_layer = layers.Dense((None, X.shape[1], 1), activation='softmax')  # Exemple, ajustez selon votre besoin
intermediate_layer = yamnet_model.layers # Exemple, ajustez selon votre besoin

# Ajouter votre propre couche de classification
output_layer = layers.Dense(2, activation='softmax')

# Créer le modèle personnalisé
model = tf.keras.Sequential([
    input_layer,
    intermediate_layer,
    output_layer
])


# Compiler le modèle avec l'optimiseur, la fonction de perte et les métriques appropriés
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

TypeError: The added layer must be an instance of class Layer. Received: layer=<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7b6e035467d0> of type <class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>.

In [ ]:
model.summary()

In [ ]:
# Entraîner le modèle
batch_size = 32
epochs = 10

# Utiliser ImageDataGenerator ou un autre moyen pour charger les données d'entraînement et de validation
train_generator = ...  # Utilisez vos propres générateurs de données
validation_generator = ...

# Définir le callback pour sauvegarder le meilleur modèle basé sur la validation
checkpoint_callback = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Entraîner le modèle
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[checkpoint_callback]
)

# Évaluation du modèle sur l'ensemble de test
test_generator = ...  # Utilisez vos propres générateurs de données pour l'ensemble de test
eval_result = model.evaluate(test_generator)
print("Test accuracy:", eval_result[1])

# Sauvegarder le modèle entraîné
model.save('trained_model.h5')
